Author: Louis Owen (https://louisowen6.github.io/)

In [1]:
import numpy as np
import re

from nlp_id.postag import PosTag

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
def get_first_last_consecutive_index(arr,main_element,element_list):
    if main_element in arr :
        first_idx = arr.index(main_element)
        
        last_idx = first_idx
        for el in arr[first_idx+1:]:
            if all(el!=element for element in element_list):
                break
            else:
                last_idx += 1
        
        return first_idx,last_idx
    else:
        return np.inf, np.inf

In [3]:
def active_to_passive(phrase_pos_list):
    phrase_list = [x[0] for x in phrase_pos_list]
    pos_list = [x[1] for x in phrase_pos_list]
    
    first_noun_phrase_loc,_ = get_first_last_consecutive_index(pos_list,'NP',['NP'])
    first_proper_noun_loc,_ = get_first_last_consecutive_index(pos_list,'NNP',['NNP'])
    first_pronoun_loc,_ = get_first_last_consecutive_index(pos_list,'PR',['PR'])
    
    subject_loc = np.min([first_noun_phrase_loc,first_proper_noun_loc,first_pronoun_loc])
    
    swapped = ' '.join(phrase_list)
    if (subject_loc!=np.inf) and (subject_loc==0): #check if sentence contains subject
        subject_loc = int(subject_loc)
        
        first_verb_loc,last_consecutive_verb_loc = get_first_last_consecutive_index(pos_list,'VB',['VB'])
        first_verb_phrase_loc,last_consecutive_verb_phrase_loc = get_first_last_consecutive_index(pos_list,'VP',['VP'])

        verb_loc = np.min([first_verb_loc,first_verb_phrase_loc])
        
        if (verb_loc!=np.inf) and (verb_loc > subject_loc): #check if sentence contains verb and it appears after subject
            verb_loc = int(verb_loc)
            verb = phrase_list[verb_loc]
            
            if verb[:2]=='me':
                verb_argmin = np.argmin([first_verb_loc,first_verb_phrase_loc])
                if verb_argmin==0:
                    last_verb_loc = int(last_consecutive_verb_loc)
                elif verb_argmin==1:
                    last_verb_loc = int(last_consecutive_verb_phrase_loc)

                if verb_loc==last_verb_loc: #checkt if there's no consecutive verbs
                    second_noun_phrase_loc,_ = get_first_last_consecutive_index(pos_list[verb_loc+1:],'NP',['NP'])
                    second_proper_noun_loc,_ = get_first_last_consecutive_index(pos_list[verb_loc+1:],'NNP',['NNP'])
                    second_noun_loc,_ = get_first_last_consecutive_index(pos_list[verb_loc+1:],'NN',['NN'])
                    second_pronoun_loc,_ = get_first_last_consecutive_index(pos_list[verb_loc+1:],'PR',['PR'])
                    
                    noun_argmin = np.argmin([second_noun_phrase_loc,second_proper_noun_loc,second_noun_loc,second_pronoun_loc])
                    if noun_argmin==0:
                        last_object_loc = verb_loc+1+get_first_last_consecutive_index(pos_list[verb_loc+1:],'NP',['NP','NNP','NN','PR','FW','NUMP','DP','UH','RP','NUM','NEG','JJ','DT','ADV'])[1]
                    elif noun_argmin==1:
                        last_object_loc = verb_loc+1++get_first_last_consecutive_index(pos_list[verb_loc+1:],'NNP',['NP','NNP','NN','PR','FW','NUMP','DP','UH','RP','NUM','NEG','JJ','DT','ADV'])[1]
                    elif noun_argmin==2:
                        last_object_loc = verb_loc+1++get_first_last_consecutive_index(pos_list[verb_loc+1:],'NN',['NP','NNP','NN','PR','FW','NUMP','DP','UH','RP','NUM','NEG','JJ','DT','ADV'])[1]
                    elif noun_argmin==3:
                        last_object_loc = verb_loc+1++get_first_last_consecutive_index(pos_list[verb_loc+1:],'PR',['NP','NNP','NN','PR','FW','NUMP','DP','UH','RP','NUM','NEG','JJ','DT','ADV'])[1]

                    if last_object_loc!=np.inf: # check if sentence contains object
                        last_object_loc = int(last_object_loc)

                        stemmed_verb = stemmer.stem(phrase_list[verb_loc])
                        
                        phrase_list_temp = phrase_list[subject_loc+1:verb_loc+1]
                        verb_loc_temp = phrase_list_temp.index(phrase_list[verb_loc])
                        phrase_list_temp[verb_loc_temp] = 'di'+stemmed_verb
                        del phrase_list_temp[verb_loc+1:last_object_loc+1]
                        phrase_list_temp.insert(0,' '.join(phrase_list[verb_loc+1:last_object_loc+1]))
                        
                        swapped = ' '.join(phrase_list_temp+[phrase_list[subject_loc]]+phrase_list[last_object_loc+1:])
                        
                        
    return swapped

In [4]:
def convert_to_passive(sentence):
    postagger = PosTag() 
    r = re.compile(r'[{}]+'.format(re.escape('.,;')))
    
    swapped_text_list = []
    for sentence in r.split(sentence):
        if sentence!=' ':
            swapped_text_list.append(active_to_passive(postagger.get_phrase_tag(sentence)))
    
    
    return ' '.join(swapped_text_list).lower().strip()

In [5]:
sentence = 'Saya ingin memainkan lagu itu dengan teman-teman saya'

convert_to_passive(sentence)

'lagu itu ingin dimain saya dengan teman-teman saya'

In [6]:
sentence = 'Sri Sultan pun mengangkat dua karung beras besar ke bagian belakang kendaraannya'

convert_to_passive(sentence)

'dua karung beras besar pun diangkat sri sultan ke bagian belakang kendaraannya'

In [7]:
sentence = 'Elektroda yang menerima elektron dari sumber arus listrik luar disebut Katoda, sedangkan elektoda yang mengalirkan elektron kembali ke sumber arus listrik luar disebut Anoda.'

convert_to_passive(sentence)

'elektron yang diterima elektroda dari sumber arus listrik luar disebut katoda sedangkan elektoda yang mengalirkan elektron kembali ke sumber arus listrik luar disebut anoda'

In [8]:
sentence = 'Pada zamannya, Faraday menyelidiki hubungan antara jumlah listrik yang mengalir dalam sel dan kuantitas kimia yang berubah di elektroda saat elektrolisis.'

convert_to_passive(sentence)

'pada zamannya hubungan diselidik faraday antara jumlah listrik yang mengalir dalam sel dan kuantitas kimia yang berubah di elektroda saat elektrolisis'

In [9]:
sentence = 'Nicholas Negroponte sendiri mengakui bahwa pertumbuhan host Internet tercepat pada kwartal ketiga 1994 terjadi di Argentina, Iran, Peru, Mesir, Filipina, Federasi Rusia, Slovenia dan Indonesia'

convert_to_passive(sentence)

'bahwa pertumbuhan host internet sendiri diaku nicholas negroponte tercepat pada kwartal ketiga 1994 terjadi di argentina iran peru mesir filipina federasi rusia slovenia dan indonesia'

In [10]:
sentence = 'Diagram RMS Titanic yang memperlihatkan pengaturan sekat kapal dengan warna merah'

convert_to_passive(sentence)

'pengaturan sekat kapal yang dilihat diagram rms titanic dengan warna merah'

In [11]:
# No Verb
sentence = '^ Thomas Beattie, seorang penumpang kelas satu, dan dua awak kapal'

convert_to_passive(sentence)

'^ thomas beattie seorang penumpang kelas satu dan dua awak kapal'

In [12]:
# Verb is not started with 'me-'
sentence = 'C (Coulomb) adalah satuan muatan listrik, dan 1 C adalah muatan yang dihasilkan bila arus 1 A (Ampere) mengalir selama 1 s'

convert_to_passive(sentence)

'c ( coulomb ) adalah satuan muatan listrik dan 1 c adalah muatan yang dihasilkan bila arus 1 a ( ampere ) mengalir selama 1 s'

In [13]:
#Already a passive sentence
sentence = 'Proses pemurnian logam kotor banyak dilakukan dalam pertambangan. Logam transisi yang kotor dapat dimurnikan dengan cara menempatkannya sebagai anode dan logam murni sebagai katode'

convert_to_passive(sentence)

'proses pemurnian logam kotor banyak dilakukan dalam pertambangan logam transisi yang kotor dapat dimurnikan dengan cara menempatkannya sebagai anode dan logam murni sebagai katode'

In [14]:
# Verb appears before subject
sentence = 'Badan sepede titanium dilapisi titanium oksida (TiO 2 )yang bersifat keras dan tidak dapat ditembus oleh oksigen atau uap air sehingga terhindar dari reaksioksida yang menyebabkan korosi'

convert_to_passive(sentence)

'badan sepede titanium dilapisi titanium oksida ( tio 2 ) yang bersifat keras dan tidak dapat ditembus oleh oksigen atau uap air sehingga terhindar dari reaksioksida yang menyebabkan korosi'

In [15]:
# Consecutive Verbs
sentence = 'Ion Cu2+ ini akan berpindah menuju keping katode sedangkan ion SO 4 2- akan menuju keping anode.'

convert_to_passive(sentence)

'ion cu2+ ini akan berpindah menuju keping katode sedangkan ion so 4 2 - akan menuju keping anode'

In [16]:
# No Subject
sentence = 'Misalnya mesin kendaraan bermotor yang terbuat dari baja umumya dilapisi kromium agar terhindar dari korosi'

convert_to_passive(sentence)

'misalnya mesin kendaraan bermotor yang terbuat dari baja umumya dilapisi kromium agar terhindar dari korosi'